In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'

query_file =  csvigo.load({path = query_fn, mode = "large"})

...Utils file loaded	


<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv	
<csv>	parsing done	


In [17]:
queries = grabNsamples(query_file, #query_file-1, nil)    --- Extracting all samples

In [21]:
maxlenq  = getMaxseq(query_file)                              --- Extracting maximum sequence length

In [22]:
queries

{
  1 : 
    {
      1 : Query
    }
  2 : 
    {
      1 : 239688
      2 : 239687
      3 : 2933
      4 : 1242
    }
  3 : 
    {
      1 : 100855
      2 : 8952
      3 : 467
    }
  4 : 
    {
      1 : 6375
      2 : 1135
    }
  5 : 
    {
      1 : 208882
      2 : 30974
      3 : 1135
    }
  6 : 
    {
      1 : 36400
      2 : 239689
    }
  7 : 
    {
      1 : 36400
      2 : 70433
    }
  8 : 
    {
      1 : 64665
      2 : 33834
    }
  9 : 
    {
      1 : 91530
      2 : 239690
    }
  10 : 
    {
      1 : 239691
      2 : 17770
    }
  11 : 
    {
      1 : 82959
      2 : 4626
      3 : 3443
    }
}


In [53]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'

x = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})

rK = 100
K = 100
nbatches = 10
nepochs = 100
print_every = 10
embed_dim = 6
learning_rate = 0.1

torch.manualSeed(69)

...Utils file loaded	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	

parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	


In [74]:
nuggets = grabNsamples(nugget_file, #nugget_file, nil)
nuggets = geti_n(nuggets, 2, #nuggets)

In [92]:
xs  = geti_n(x, 2, #x-1)
preds = torch.round(torch.rand(#xs))
predsummary = buildPredSummary(preds, xs)

In [93]:
preds[1], preds[2], preds[3], preds[4]

0	1	1	1	


In [94]:
xs[1], xs[2], xs[3], xs[4]

{
  1 : 21 16 20 2 1 4 11 22 19 9 14 20 25 24 23 24 8 5 7 17 3 12
}
{
  1 : 235 218 129 54 23 248 62 249 119 224 245 23 57 262 5 238 174 242 216 265 83 231 266 250 257 54 265 62 221 263 258 223 244 247 1 229 224 265 241 100 220 1 243 264 3 129 1 272 1 228 264 3 236 132 237 255 225 263 24 230 261 1 228 232 240 270 3 129 1 24 260 1 252 240 270 3 236 54 265 239 24 67 28 227 266 250 257 192 222 1 129 83 269 253
}
{
  1 : 329 334 332 23 335 129 333 330 119 321 63 336
}
{
  1 : 20 437 438 119 432 192 24 359 436 434 3 433 129 431 224 440 429 140 3 435 83 430 439 441
}


In [99]:
nuggets[1]

{
  1 : 1963
  2 : 1030
  3 : 165
  4 : 474
  5 : 1563
  6 : 3
  7 : 24
  8 : 4309
  9 : 23
  10 : 111
  11 : 986
  12 : 2206
  13 : 129
}


In [100]:
predsummary[1], predsummary[2], predsummary[3], predsummary[4]

{
  1 : 0
}
{
  1 : 235 218 129 54 23 248 62 249 119 224 245 23 57 262 5 238 174 242 216 265 83 231 266 250 257 54 265 62 221 263 258 223 244 247 1 229 224 265 241 100 220 1 243 264 3 129 1 272 1 228 264 3 236 132 237 255 225 263 24 230 261 1 228 232 240 270 3 129 1 24 260 1 252 240 270 3 236 54 265 239 24 67 28 227 266 250 257 192 222 1 129 83 269 253
}
{
  1 : 329 334 332 23 335 129 333 330 119 321 63 336
}
{
  1 : 20 437 438 119 432 192 24 359 436 434 3 433 129 431 224 440 429 140 3 435 83 430 439 441
}


In [103]:
nuggets

{
  1 : 
    {
      1 : 1963
      2 : 1030
      3 : 165
      4 : 474
      5 : 1563
      6 : 3
      7 : 24
      8 : 4309
      9 : 23
      10 : 111
      11 : 986
      12 : 2206


      13 : 129
    }
  2 : 
    {
      1 : 15734
      2 : 146
      3 : 24
      4 : 1135
      5 : 193
      6 : 3
      7 : 146
      8 : 1030
      9 : 57
      10 : 406
      11 : 1575
      12 : 192
      13 : 57
      14 : 129
      15 : 3771
      16 : 177
      17 : 185
      18 : 4581
      19 : 24
      20 : 7475
      21 : 6029


      22 : 7962
    }
  3 : 
    {
      1 : 2108
      2 : 47
      3 : 81
      4 : 1919
      5 : 157
      6 : 1243
      7 : 450
      8 : 214
      9 : 620
      10 : 585
      11 : 4617
      12 : 685
    }
  4 : 
    {
      1 : 1057
      2 : 165
      3 : 3
      4 : 2825
      5 : 474
      6 : 83
      7 : 4522
      8 : 1624
      9 : 1694
    }
  5 : 
    {
      1 : 5895
      2 : 703
      3 : 1864
      4 : 83
      5 : 2618
      6 : 28
      7 : 575
      8 : 49
      9 : 192
      10 : 24
      11 : 1135
      12 : 1595
    }
  6 : 
    {
      1 : 54
      2 : 1887
      3 : 2077
      4 : 1249
      5 : 57
      6 : 4671
      7 : 2555
      8 : 3
      9 : 24
      10 : 3771
      11 : 869
      12 : 72
      13 : 2975
      14 : 1135
    }
  7 : 
    {
      1 : 54
      2 : 1887
      3 : 62


      4 : 4382
      5 : 83
      6 : 214
      7 : 479
      8 : 48
      9 : 165
      10 : 1
      11 : 656
      12 : 3568
    }
  8 : 
    {
      1 : 2108
      2 : 1817
      3 : 4614
      4 : 57
      5 : 2596
      6 : 185
      7 : 2604
      8 : 475
      9 : 24
      10 : 1135
    }
  9 : 
    {
      1 : 54
      2 : 4368
      3 : 1621
      4 : 185
      5 : 3367
    }
  10 : 
    {
      1 : 54
      2 : 1621
      3 : 7491
      4 : 641
      5 : 24
      6 : 5895
      7 : 23
      8 : 42


      9 : 24
      10 : 23
      11 : 1486
      12 : 174
    }
  11 : 
    {
      1 : 7307
      2 : 474
      3 : 83
      4 : 359
      5 : 355
      6 : 28
      7 : 11166
    }
  12 : 
    {
      1 : 165
      2 : 474
      3 : 7954
      4 : 575
      5 : 57
      6 : 134
      7 : 2077
      8 : 151
      9 : 24
      10 : 1595
      11 : 23
      12 : 24
      13 : 2543
    }
  13 : 
    {
      1 : 24
      2 : 1817
      3 : 1864
      4 : 2548
      5 : 177
      6 : 194
      7 : 185
      8 : 4348
      9 : 3
      10 : 36
    }
  14 : 
    {
      1 : 2012
      2 : 83
      3 : 24


...

      10 : 36
      11 : 2016
      12 : 3
      13 : 129
      14 : 221
      15 : 263
    }
  118 : 
    {
      1 : 1564
      2 : 6739
      3 : 185
    }
  119 : 
    {
      1 : 1835
      2 : 2656
      3 : 185
      4 : 7878
      5 : 263
      6 : 24
    }
  120 : 
    {
      1 : 54
      2 : 57
      3 : 10544
    }
  121 : 
    {
      1 : 14768
      2 : 407
      3 : 298
      4 : 1692
      5 : 119
      6 : 321
      7 : 336
      8 : 63
      9 : 57
      10 : 406
    }
  122 : 
    {
      1 : 23
      2 : 24
      3 : 1444
      4 : 24
      5 : 1595
      6 : 1
      7 : 1444
      8 : 3
      9 : 459
    }
  123 : 
    {
      1 : 54
      2 : 638
      3 : 23
      4 : 57
      5 : 8
      6 : 882
      7 : 47
      8 : 83
      9 : 311
      10 : 28
      11 : 112
      12 : 23
      13 : 2781
      14 : 23
      15 : 10669
      16 : 2766
      17 : 301
      18 : 24
    }
  124 : 
    {
      1 : 8028
      2 : 8447
      3 : 1623
      4 : 83
      5 : 7517
  

 1 : 16740
      2 : 1979
      3 : 82
      4 : 298
      5 : 4250
      6 : 57
      7 : 19437
      8 : 83
      9 : 129
      10 : 67
      11 : 3141
      12 : 5086
      13 : 947
      14 : 83
      15 : 1950
      16 : 24
      17 : 23
      18 : 24
      19 : 1135
    }
  126 : 
    {
      1 : 83
      2 : 2307
      3 : 8105
      4 : 177
      5 : 72
      6 : 147
      7 : 57
      8 : 1540
      9 : 1
      10 : 507
      11 : 83
    }
  127 : 
    {
      1 : 768
      2 : 321
      3 : 812
      4 : 336
      5 : 189
      6 : 23
      7 : 24
      8 : 1311
      9 : 27
      10 : 407
      11 : 10040
      12 : 316
      13 : 7516
      14 : 868
      15 : 694
      16 : 5353
    }
  128 : 
    {
      1 : 14170
      2 : 474
      3 : 2796
      4 : 205
      5 : 641
      6 : 11306
    }
  129 : 
    {
      1 : 54
      2 : 1
      3 : 16740
      4 : 214
      5 : 1088
      6 : 142
      7 : 140
      8 : 4310
      9 : 83
      10 : 700
    }
  130 : 
    {
      

      11 : 1978
      12 : 6029
      13 : 316
      14 : 177
      15 : 19808
      16 : 24
      17 : 111
      18 : 2543
      19 : 3
      20 : 5068
      21 : 1
      22 : 1859
    }
  131 : 
    {
      1 : 13664
      2 : 185
      3 : 3
      4 : 57
      5 : 23
      6 : 721
      7 : 719
      8 : 731
      9 : 57
      10 : 3367
      11 : 1542
      12 : 1111
      13 : 3
      14 : 575
      15 : 1978
      16 : 83
      17 : 36
      18 : 26261
      19 : 1703
      20 : 72
      21 : 62
      22 : 57
      23 : 83
      24 : 24
      25 : 111
      26 : 1817
      27 : 67
      28 : 2239
    }
  132 : 
    {
      1 : 18519
      2 : 9361
      3 : 192
      4 : 224
      5 : 8105
      6 : 3
      7 : 4420
      8 : 4813
      9 : 1362
      10 : 23
      11 : 57
      12 : 2543
      13 : 3
      14 : 13397
      15 : 3
    }
  133 : 
    {
      1 : 1295
      2 : 23
      3 : 24
      4 : 1486
      5 : 174
      6 : 59
      7 : 82
      8 : 185
      9 : 1089
     

   {
      1 : 132
      2 : 185
      3 : 177
      4 : 3
      5 : 24
      6 : 1312
      7 : 23
      8 : 24
      9 : 1135
      10 : 18515
      11 : 24
      12 : 23
      13 : 7475
      14 : 183
      15 : 292
      16 : 57
      17 : 1595
      18 : 23
      19 : 57
      20 : 5794
      21 : 8051
      22 : 57
      23 : 10549
      24 : 51
      25 : 23
      26 : 105
      27 : 666
      28 : 103
      29 : 57
    }
  135 : 
    {
      1 : 82
      2 : 185
      3 : 177
      4 : 16740
      5 : 3215
      6 : 1
      7 : 9330
      8 : 24
      9 : 1732
      10 : 3
      11 : 24
      12 : 24
      13 : 7475
      14 : 1633
      15 : 3
      16 : 2656
      17 : 83
      18 : 24
      19 : 3
      20 : 24
      21 : 2798
    }
  136 : 
    {
      1 : 24
      2 : 1428
      3 : 23
      4 : 1609
      5 : 3296
      6 : 57
      7 : 1540
      8 : 3
      9 : 7087
      10 : 83
      11 : 57
      12 : 1845
      13 : 1892
      14 : 24
      15 : 1486
      16 : 174


      12 : 1
      13 : 24
      14 : 4127
      15 : 192
      16 : 1199
      17 : 4398
      18 : 83
      19 : 144
      20 : 194
    }
  138 : 
    {
      1 : 768
      2 : 321
      3 : 336
      4 : 1194
      5 : 224
      6 : 177
      7 : 1557
      8 : 24
      9 : 83
      10 : 24
      11 : 94
      12 : 129
      13 : 1692
      14 : 4622
      15 : 1200
      16 : 1892
      17 : 292
      18 : 2069
      19 : 1030
      20 : 23
      21 : 3028
      22 : 1816
    }
  139 : 
    {
      1 : 5499
      2 : 24
      3 : 72
      4 : 1967
      5 : 83
      6 : 36
      7 : 2
      8 : 83
      9 : 57
      10 : 4354
      11 : 3
      12 : 24
      13 : 3584
      14 : 898
    }
}


In [121]:
out  = grabNsamples(xs, 1, #xs)

In [120]:
predsummary[2][1]

235 218 129 54 23 248 62 249 119 224 245 23 57 262 5 238 174 242 216 265 83 231 266 250 257 54 265 62 221 263 258 223 244 247 1 229 224 265 241 100 220 1 243 264 3 129 1 272 1 228 264 3 236 132 237 255 225 263 24 230 261 1 228 232 240 270 3 129 1 24 260 1 252 240 270 3 236 54 265 239 24 67 28 227 266 250 257 192 222 1 129 83 269 253	


In [114]:
Tokenize(getLastK(predsummary, 100))

{
  0 : 55
  1031 23 24 945 2849 3 28 24 336 2847 3350 1 194 474 838 177 2015 868 : 1
  19641 7092 813 1592 119 36 5020 1516 36 1161 2105 125 57 2943 24 5042 119 95 : 1
  7962 7966 1095 4581 810 1136 24 4548 1789 1564 953 28 57 988 58 7475 487 7711 263 7957 54 62 1095 536 23 24 157 62 7584 705 4548 1789 83 810 3 24 1633 54 7962 : 1
  6976 6977 112 5154 82 4995 119 5649 83 810 8967 946 57 4141 16237 59 103 72 24 119 57 5154 : 1
  7306 57 57 6394 57 : 1
  2990 67 42 572 570 204 203 816 868 23 24 598 : 1
  82 67 1111 224 116 34 : 1
  54 7962 7966 196 810 361 360 4548 1789 139 24 1828 475 3 57 9132 9463 140 : 1
  13418 299 20 15714 470 1764 1896 192 111 62 3096 42 23 1643 67 28 57 5142 : 1
  24 5980 298 83 1459 24 2 23 5980 83 6207 450 298 1459 1764 105 214 24 2 83 8509 1 132 47 204 203 1110 875 7185 157 42 1953 62 138 57 277 415 176 407 6940 3 24 11227 456 1191 59 27 1919 120 1916 24 2582 685 23 146 1085 140 5427 1 82 83 2078 5175 3 42 23 24 1643 67 4223 165 474 3 1284 725 475 57 10819 11

In [107]:
rscore = rougeRecall({predsummary[2]}, nuggets, K)
pscore = rougePrecision({predsummary[2]}, nuggets, K)

2	0	
1	0	
1	0	
1	0	
4	0	
1	0	
1	0	
2	0	
2	0	
4	0	
1	0	
1	0	
9	0	
7	0	
3	0	
1	0	
1	0	
1	0	
3	0	
4	0	
19	0	
2	0	
2	0	
2	0	
1	0	
4	0	
1	0	
2	0	
1	0	
1	0	
1	0	
4	0	
1	0	
1	0	
1	0	
2	0	
2	0	
12	0	
2	0	
1	0	
5	0	
3	0	
1	0	
14	0	
5	0	
1	0	
1	0	
2	0	
1	0	
2	0	
2	0	
2	0	
2	0	
1	0	
1	0	
1	0	
5	0	


1	0	
1	0	
1	0	
1	0	
3	0	
1	0	
1	0	
1	0	
5	0	
1	0	
1	0	
1	0	
1	0	
1	0	
2	0	
4	0	
1	0	
26	0	
3	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
20	0	
6	0	
3	0	
1	0	
1	0	
1	0	
3	0	
3	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
3	0	
2	0	
1	0	
2	0	
2	0	
1	0	
2	0	
2	0	
1	0	
3	0	
1	0	
3	0	
3	0	
1	0	
1	0	
1	0	
10	0	
1	0	
3	0	
4	0	
60	0	
2	0	
2	0	
1	0	
2	0	
1	0	
1	0	
1	0	
2	0	
2	0	
2	0	
1	0	
1	0	
4	0	
6	0	
1	0	
2	0	
13	0	
2	0	
2	0	
1	0	
1	0	
5	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
0	1	
1	0	
3	0	
1	0	
2	0	
1	0	
5	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
2	0	
61	0	
2	0	
1	0	
2	0	
2	0	
10	0	
1	0	
1	0	
1	0	
1	0	
1	0	
4	0	
3	0	
1	0	
4	0	
1	0	
1	0	
1	0	
1	0	
14	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
4	0	
1	0	
1	0	
1	0	
1	0	
1	0	
5	0	
1	0	
1	0	
1	0	
10	0	
1	0	
1	0	
3	0	
2	0	
1	0	
3	0	
5	0	
1	0	
1	0	
1	0	
3	0	
1	0	
2	0	
10	0	
2	0	


2	0	
1	0	
9	0	
1	0	
6	0	
1	0	
1	0	
1	0	
9	0	
5	0	
1	0	
72	0	
1	0	
1	0	
6	0	
4	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
2	0	
1	0	
74	0	
4	0	
2	0	
1	0	
3	0	
1	0	
3	0	
3	0	
1	0	
1	0	
3	0	
1	0	
1	0	
4	0	
10	0	
1	0	
2	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
2	0	
1	0	
4	0	
1	0	
20	0	
1	0	
1	0	
3	0	
1	0	
1	0	
3	0	
4	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
3	0	
4	0	
1	0	
1	0	
3	0	
1	0	
1	0	
3	0	
2	0	
6	0	
1	0	
2	0	
2	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
11	0	
1	0	
1	0	
1	0	
8	0	
1	0	
2	0	
1	0	
2	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
5	0	
1	0	
1	0	
1	0	
1	0	
6	0	
1	0	
2	0	
8	0	
1	0	
3	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
14	0	
1	0	
5	0	
1	0	
3	0	
1	0	
1	0	
1	0	
2	0	
2	0	
1	0	
2	0	
1	0	
1	0	
1	0	
2	0	
1	0	
2	0	
1	0	
1	0	
1	0	
3	0	
2	0	
2	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
7	0	
1	0	
2	0	
2	0	
5	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
2	0	
2	0	
1	0	
5	0	


1	0	
3	0	
1	0	
1	0	
2	0	
1	0	
2	0	
2	0	
3	0	
2	0	
7	0	
1	0	
6	0	
1	0	
1	0	
2	0	
6	0	
8	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
18	0	
1	0	
2	0	
1	0	
1	0	
5	0	
1	0	
1	0	
10	0	
3	0	
1	0	
1	0	
2	0	
1	0	
2	0	
1	0	
2	0	
2	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
17	0	
2	0	
36	0	
2	0	
6	0	
1	0	
2	0	
1	0	
1	0	
2	0	
1	0	
5	0	
2	0	
3	0	
1	0	
4	0	
1	0	
1	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
3	0	
10	0	
2	0	
1	0	
45	0	
3	0	
1	0	
1	0	
1	0	
1	0	
1	0	
1	0	
3	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
2	0	
2	0	
1	0	
1	0	
9	0	
1	0	
1	0	
1	0	
1	0	
10	0	
1	0	
12	0	
3	0	
1	0	
1	0	
1	0	
1	0	
1	0	
13	0	
3	0	
1	0	
3	0	
31	0	
3	0	
8	0	
5	0	
3	0	
2	0	
2	0	
1	0	
1	0	
2	0	
3	0	
1	0	
1	0	
3	0	
4	0	
1	0	
1	0	
1	0	
1	0	
3	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
3	0	
146	0	
1	0	
8	0	
1	0	
2	0	
1	0	
3	0	
7	0	
1	0	
2	0	
1	0	
1	0	
1	0	
3	0	
7	0	
1	0	
1	0	
1	0	
2	0	
1	0	
2	0	
2	0	
1	0	
1	0	
5	0	
1	0	
4	0	
1	0	
1	0	
1	0	
3	0	
1	0	
2	0	
1	0	
1	0	
2	0	
2	0	
2	0	
1	0	
2	0	
1	0	
2	0	
1	0	
82	0	
2	0	
1	0	
1	0	
1	0	
4	0	
12	0	



1	0	
1	0	
5	0	
6	0	
6	0	
4	0	
5	0	
3	0	
4	0	
1	0	
2	0	
1	0	
1	0	
1	0	
1	0	
1	0	
16	0	
7	0	
4	0	
1	0	
1	0	
11	0	
4	0	
12	0	
2	0	
1	0	
1	0	
1	0	
2	0	
1	0	
0	2129	
0	1	


In [98]:
rscore, pscore, fscore

0	0	0	


In [105]:
function rougeRecall(pred_summary, ref_summaries, K)
    pred_summary = getLastK(pred_summary, K)
    rsd = Tokenize(ref_summaries)
    sws = Tokenize(pred_summary)
    for k, v in pairs(rsd) do
        if sws[k] == nil then
            sws[k] = 0
        end
    end

    for k, v in pairs(sws) do
        if rsd[k] == nil then
            rsd[k] = 0 
        end
    end
    num = 0.
    den = 0.
    for k, v in pairs(rsd) do
        print(rsd[k], sws[k])
        num = num + math.min(rsd[k], sws[k])
        den = den + rsd[k]
    end
    print(num, den)
    return num/den
end
---- Precision
function rougePrecision(pred_summary, ref_summaries, K)
    pred_summary = getLastK(pred_summary, K)
    rsd = Tokenize(ref_summaries)
    sws = Tokenize(pred_summary)
    for k, v in pairs(rsd) do
        if sws[k] == nil then
            sws[k] = 0
        end
    end
    for k, v in pairs(sws) do
        if rsd[k] == nil then
            rsd[k] = 0
        end
    end
    num = 0.
    den = 0.
    for k, v in pairs(rsd) do
        num = num + math.min(rsd[k], sws[k])
        den = den + sws[k]
    end
    print(num, den)
    return num/den
end
---- F1
function rougeF1(pred_summary, ref_summaries, K) 
    rnp = rougeRecall(pred_summary, ref_summaries, K)
    rnr = rougePrecision(pred_summary, ref_summaries, K)
    --- Had to add this line b/c f1 starts out at 0
    f1 = (rnr > 0) and (2. * rnp * rnr ) / (rnp + rnr) or 0
    return f1
end